<a href="https://colab.research.google.com/github/0xfffddd/Coding/blob/main/MGMT687_HW3_Lichen_Mao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Q1.a.**

Methdology:

To assess whether the treatment and control groups are comparable by conducting a randomization check as stated in the problem, we will try solve by the approach discussed in class.

Firstly, we will verify that the sample sizes across the control and treatment groups are balanced, to confirm that the samples are in a similar ratio(In code we will use H0= 1/3 of whole data for each treatment/control group - under 5% significant level - which means the samples for each groups are evenly distributed withou missing, this should be corredponding to the imformation of 'Description of Variants'in assignments).

Secondly, we will compare all variables that should not be affected by the treatment. Statistical tests will be used to examine whether there are significant differences across groups.In the code we will assume H0 = for each each variable in each treatment group, have no systematic difference with control group - under 5% significant level. If most variables show no significant differences, the groups can be considered comparable and the randomization can be viewed as valid, since a small number of significant results may occur by chance at the 5% significance level.

Next is the code for implementing this methodology:


In [3]:
# Code for Q1.a

#step1 import libiary
import pandas as pd
import numpy as np
from scipy import stats

#step2 load data and check data integrity
df = pd.read_csv("data_ctr.csv")
print("Shape:", df.shape)
print("Columns:", df.columns.tolist())
print("\nMissing values per column:\n", df.isna().sum())
df["variant"] = df["variant"].astype(int)
df["regDate_dt"] = pd.to_datetime(df["regDate"], format="%d-%m-%Y", errors="coerce")
df["regDate_ord"] = df["regDate_dt"].map(lambda x: x.toordinal() if pd.notna(x) else np.nan)

#step3 sample ratio check
#assume H0: observed split matches expected split (equal 1/3,1/3,1/3)
#under 5% significant level
counts = df["variant"].value_counts().sort_index()
print("\nGroup sizes (variant counts):\n", counts)
total_n = counts.sum()
expected = np.array([total_n/3, total_n/3, total_n/3])
observed = counts.values
chi2_stat, chi2_p = stats.chisquare(f_obs=observed, f_exp=expected)
print("\nSRM Chi-square test:")
print(f"  chi2 = {chi2_stat:.4f}, p-value = {chi2_p:.6g}")
print("  (If p > 0.05, no evidence of SRM / sample ratio mismatch.)")

#step4 Ramdomization check by both ANOVA and pairwise
#assume H0:  for each each variable in each treatment group, have no systematic difference with control group
#under 5% significant level
baseline_vars = [
    "age", "wage", "eduLv", "regDate_ord",
    "friendSum", "followSum", "groupSum", "tagSum",
    "loginFreq", "actTime", "msgSent", "postFreq", "shareFreq",
    "clickCount", "likeCount", "cmtCount"
]
summary = df.groupby("variant")[baseline_vars].agg(["mean", "std"])
print("\nBaseline summary by variant (mean/std):")
display(summary)

#ANOVA p-values test for whole dataset
anova_results = []
for v in baseline_vars:
    g0 = df.loc[df["variant"] == 0, v].dropna()
    g1 = df.loc[df["variant"] == 1, v].dropna()
    g2 = df.loc[df["variant"] == 2, v].dropna()
    f_stat, p_val = stats.f_oneway(g0, g1, g2)
    anova_results.append((v, f_stat, p_val))
anova_df = pd.DataFrame(anova_results, columns=["variable", "F_stat", "p_value"]).sort_values("p_value")
print("\nANOVA balance checks (smaller p means more imbalance):")
display(anova_df)
alpha = 0.05
num_sig = (anova_df["p_value"] < alpha).sum()
print(f"\nAt alpha={alpha}, significant baseline differences: {num_sig} out of {len(baseline_vars)}")

#pairwise t-tests vs control (0 vs 1, 0 vs 2) for each baseline var
pairwise = []
for v in baseline_vars:
    c = df.loc[df["variant"] == 0, v].dropna()
    a = df.loc[df["variant"] == 1, v].dropna()
    b = df.loc[df["variant"] == 2, v].dropna()
    t01, p01 = stats.ttest_ind(a, c, equal_var=False, nan_policy="omit")
    t02, p02 = stats.ttest_ind(b, c, equal_var=False, nan_policy="omit")
    pairwise.append((v, t01, p01, t02, p02))
pairwise_df = pd.DataFrame(
    pairwise,
    columns=["variable", "t_A_vs_C", "p_A_vs_C", "t_B_vs_C", "p_B_vs_C"]
).sort_values(["p_A_vs_C", "p_B_vs_C"])
print("\nPairwise balance checks:")
display(pairwise_df)

#judgement
print("If p-value above (each variable or p_A_vs_C or p_B_vs_C) is larger then 0.05, that means no reject its H0, which means this variable have no systematic difference. ")



Shape: (30000, 18)
Columns: ['variant', 'age', 'wage', 'eduLv', 'regDate', 'friendSum', 'followSum', 'groupSum', 'tagSum', 'loginFreq', 'actTime', 'msgSent', 'postFreq', 'shareFreq', 'clickCount', 'likeCount', 'cmtCount', 'CTR']

Missing values per column:
 variant       0
age           0
wage          0
eduLv         0
regDate       0
friendSum     0
followSum     0
groupSum      0
tagSum        0
loginFreq     0
actTime       0
msgSent       0
postFreq      0
shareFreq     0
clickCount    0
likeCount     0
cmtCount      0
CTR           0
dtype: int64

Group sizes (variant counts):
 variant
0    10000
1    10000
2    10000
Name: count, dtype: int64

SRM Chi-square test:
  chi2 = 0.0000, p-value = 1
  (If p > 0.05, no evidence of SRM / sample ratio mismatch.)

Baseline summary by variant (mean/std):


age                    wage                eduLv            \
            mean       std          mean          std    mean       std   
variant                                                                   
0        36.5444  8.947460  22996.299236  6048.853151  2.7397  0.980938   
1        36.6069  8.970408  22999.459805  5980.373773  2.7294  0.976151   
2        36.3334  9.013166  22946.900517  5958.556345  2.7312  0.966561   

           regDate_ord             friendSum              ... postFreq  \
                  mean         std      mean         std  ...     mean   
variant                                                   ...            
0        736664.341178  345.532488  440.2305  477.794508  ...   5.8698   
1        736666.564183  348.091544  440.3239  472.050556  ...   5.8257   
2        736660.584915  351.915098  430.4334  455.597022  ...   5.7909   

                  shareFreq            clickCount            likeCount  \
              std      mean        std       mean        std      mean   
variant                                                                  
0        7.014454   14.6064  17.356670    53.2261  62.920543  113.5721   
1        7.007070   14.5141  16.895264    53.4171  59.524866  110.2054   
2        7.215676   14.5621  17.110191    53.4387  60.120308  112.6757   

                     cmtCount              
                std      mean         std  
variant                                    
0        175.835702  194.3184  237.107307  
1        158.489706  190.1909  226.058842  
2        173.650508  189.3077  217.187724  

[3 rows x 32 columns]


ANOVA balance checks (smaller p means more imbalance):


,variable,F_stat,p_value
0,age,2.548566,0.078211
5,followSum,2.191970,0.111714
4,friendSum,1.471212,0.229664
15,cmtCount,1.389170,0.249298
14,likeCount,1.058141,0.347114
7,tagSum,1.024913,0.358840
10,msgSent,0.802834,0.448067
3,regDate_ord,0.458791,0.632055
8,loginFreq,0.380435,0.683567
6,groupSum,0.334634,0.715603



At alpha=0.05, significant baseline differences: 0 out of 16

Pairwise balance checks:


,variable,t_A_vs_C,p_A_vs_C,t_B_vs_C,p_B_vs_C
14,likeCount,-1.422218,0.154979,-0.362726,0.716814
10,msgSent,1.265630,0.205660,0.471777,0.637091
15,cmtCount,-1.259916,0.207714,-1.558326,0.119172
7,tagSum,-1.238272,0.215630,-0.001032,0.999176
2,eduLv,-0.744287,0.456712,-0.617227,0.537092
9,actTime,-0.632485,0.527077,-0.403562,0.686539
6,groupSum,0.560002,0.575485,0.796605,0.425690
0,age,0.493297,0.621808,-1.661396,0.096650
5,followSum,0.474719,0.634992,-1.528839,0.126320
11,postFreq,-0.444793,0.656474,-0.784043,0.433024


If p-value above (each variable or p_A_vs_C or p_B_vs_C) is larger then 0.05, that means no reject its H0, which means this variable have no systematic difference. 


# **Q1.b.**


Based on the code results in (a), the interpret of whether the randomization procedure can be considered valid from three aspects:

1. **Sample size balance:** The sample sizes across the three groups are perfectly balanced, with 10,000 users in each group. In (a) result, The sample ratio mismatch test yields a p-value of 1, indicating no evidence of allocation errors. In addition, there is no missing data in the dataset. Therefore, the overall background for randomization appears appropriate and well-implemented.

2.  **ANOVA results** : The ANOVA results show that none of the baseline variables are statistically significant at the 5% level (In (a) result, all variable's p_value are larger than 0.05 in 'ANOVA balance checks'table ). This means there is no overall difference across the groups simultaneously, suggesting that the groups are comparable on observed pre-treatment characteristics.

3. **Pairwise comparison results:** Pairwise tests between the treatment and control groups also show no significant differences between each treatment group and the control group in the dataset (in (a) result, either all the variable in p_A_vs_C or p_A_vs_C 's p-value are larger than 0.05 in 'Pairwise balance checks' table). These findings further suggest that the three groups are comparable before the experiment.

**Conclusion: Based on the above randomization checks, the treatment and control groups appear comparable. Therefore, the randomization procedure can be considered valid.**

# **Q2**
# **Part1. Analyze the experimental data and treatment effects**

We can design a data interpretation pipeline to analyze the experimental data and identify the treatment effect for each product decision. It starts with calculating CTR, then conducting a t-test for each treatment group, and finally confirming whether the experiment can support the treatment effect.

Firstly, we calculate the CTR (mean) for each group:

In [7]:
#CTR
df.groupby("variant")["CTR"].mean()

,CTR
variant,
0,0.057826
1,0.063204
2,0.072567


From the assignment, we know that 0 is the control group, while 1 is Treatment A and 2 is Treatment B. **Based on the CTR results, Treatment B > Treatment A > Control Group.** If we consider the treatment effect as the difference in CTR, Treatment B is around 0.0147 higher than the control, while Treatment A is about 0.0054 higher than the control, based on the results above. Also, considering that n = 10,000 for each group, we can assume there is **strong statistical power**.

This suggests that both treatments have higher CTR than the control group. However, we are not yet confident because we do not know whether this difference is due to random noise or represents a statistically significant difference.


To determine whether these differences are random noise or statistically significant, we assume the null hypothesis (H0): the mean CTR for each treatment group is the same as the control group. We then calculate the test statistics (t and p-values):



In [8]:
#Statastic Calculation
#get CTR
from scipy import stats
control_ctr = df[df["variant"]==0]["CTR"]
A_ctr = df[df["variant"]==1]["CTR"]
B_ctr = df[df["variant"]==2]["CTR"]
#run t-test and p value calculation function
t_A, p_A = stats.ttest_ind(A_ctr, control_ctr, equal_var=False)
t_B, p_B = stats.ttest_ind(B_ctr, control_ctr, equal_var=False)
#output
print("A vs Control: t =", t_A, "p =", p_A)
print("B vs Control: t =", t_B, "p =", p_B)

A vs Control: t = 8.749325921472696 p = 2.3142504934962443e-18
B vs Control: t = 22.521719571420654 p = 6.733080148053092e-111


As shown above,


*   for Treatment A, t≈8.749, p value≈2e-18
*   for Treatment B, t≈22.52, p value≈7e-111



The t-test shows how strong the difference is, while the p-value indicates the significance level of the difference. Assuming a significance level of 5% (0.05), all p-values are far below this threshold, which means we can reject H0. As a result, we can confirm that **both Treatment A and Treatment B have higher CTR than the control group at a statistically significant level.** Treatment B shows an even stronger treatment effect.

However, this conclusion still has its limitations, which is --

# **Part2. discuss whether the results can be concluded with 100% certainty and explain why or why not**
Despite the strong results, **we still can not conclude the result with 100% certainty.**

Even when the results are statistically significant, there remains a possibility of a Type I error (false positive), meaning that the observed difference could occur by chance. In this dataset, we only conducted sampling once. This means that results from another experiment with different samples could lead to different outcomes—we are not fully certain whether our observation has captured the true CTR for each group.

Therefore, while the evidence strongly suggests that both treatments improve CTR, we cannot conclude with 100% certainty. Instead, we can conclude that the improvements are highly likely and statistically supported based on the observed data.


# **Q3.a**

Yes, the decision is subject to potential statistical errors, such as Type I and Type II errors.

**Type1 error:** In Q2 we used 5% significant level to judge whether reject the hypothesis. From the defination of type I error, the rate of it is also 5%, which means we have 5% chance to rejecting the null hypothesis, when it is ture and we should not.

**Type2 error:** we can assume the observed Treatment - Control differences represent the true effects and evaluate the probability that the test statistic falls outside the rejection region.With the defined of type II error beta=P(fail to reject H0 under H1 is ture)=1- power. Using the results from Question 2 Part 1 (t ≈ 8.75 for Treatment A and t ≈ 22.52 for Treatment B), we can calculate Type II error. As shown in the code below, the results are. As the result of calculation (shown in the below coding),


*   Treatment A beta ≈ 6e-12
*   Treatment B beta ≈ 3e-94

Both beta are extremely small.

This implies that the statistical power can be considered as 1 for both comparisons, indicating a negligible probability of failing to detect the true treatment effects.



In [11]:
#code to calculate type 2 error
from scipy.stats import norm
#input alpha = 0.05 and calculate z crit (should≈1.96 under 0.05 significant level)
alpha = 0.05
z_crit = norm.ppf(1 - alpha/2)
print("Critical value (z_crit):", z_crit)
#use T observed in question 2
t_A = 8.749325921472696
t_B = 22.521719571420654
#calculate Type II error beta
beta_A = norm.cdf(z_crit - t_A) - norm.cdf(-z_crit - t_A)
beta_B = norm.cdf(z_crit - t_B) - norm.cdf(-z_crit - t_B)
print("Beta for Treatment A:", beta_A)
print("Beta for Treatment B:", beta_B)
#and calculate statistical power
power_A = 1 - beta_A
power_B = 1 - beta_B
print("Power for Treatment A:", power_A)
print("Power for Treatment B:", power_B)

Critical value (z_crit): 1.959963984540054
Beta for Treatment A: 5.631527830269382e-12
Beta for Treatment B: 3.0203705944235076e-94
Power for Treatment A: 0.9999999999943685
Power for Treatment B: 1.0


# **Q3.b**

I think there are several possible strategies to mitigate Type I and Type II errors in general experimental settings.

To mitigate Type I errors (false positives), experiment can use a more conservative significance level. In my Q2 case, an example might be reduce the significance leve from 5% to 0.5% or even 0.05%, as we found that p value are extremely small (lower than 1e-6 after calculation). We should carefully control for multiple testing, and ensure proper randomization and clean data collection so that results are not driven by bias or noise.Another idea is sampling over and over again decrease the effect of different sampling. If the sampleing is limited, separate the data with calculation (test) and validation may also help improve reliability.

To mitigate Type II errors (false negatives), increasing sample size is often the most effective approach, as larger samples improve statistical power and make true effects easier to detect. In the assignment case, the sample amount n =10000 seems large enough, and we found the power for each treatment is very close to 1, but still we can increase the sample or user more detailed metrics (variables). In addition, improving measurement accuracy, refining experimental design, and allowing sufficient experiment duration can further enhance reliability.

In practice, there is always a trade-off between the two types of errors, time, cost, so experimenters should choose an appropriate balance based on the business context and decision risk.

Q3.C

Recall the result in Q2:


*    Treatment B is around 0.0147 higher than the control, while Treatment A is about 0.0054 higher than the control, based on the results above
*   A vs Control: t = 8.749325921472696 p = 2.3142504934962443e-18

    B vs Control: t = 22.521719571420654 p = 6.733080148053092e-111

It is clear that the company should adopt product B, which is 'Provide a preview of the content and a thumbnail along with the share link.'Because in treatment, product B shows the strongest effect and the most convincing statistical evidence.

Firstly, both Treatment A and Treatment B achieve higher CTR than the control group, with extremely small p-values. This suggests that we can be highly confident (though not 100% certain as stated in Q2) that both designs will improve user interaction and potentially increase business value. Since Treatment B shows the largest improvement in CTR, it performs better than Treatment A.

Secondly, B also has very strong statistical power and an extremely low Type II error rate. This indicates that the observed effect is highly reliable and unlikely to be driven by random variation, providing stronger support for adopting product B.

In real business, even though Product B performs best statistically, the company should still evaluate implementation costs and potential risks before making the final decision. A final product decision should balance both statistical evidence and business considerations.